! description


given a csv or such table, where rows are samples and columuns are genes.
create a h5ad file where the count values are the .X matrix, the gene are in .var as "gene_hgnc" and the sample are in .obs as "sample_id"

In [1]:
# import
import anndata
import pandas as pd
import os
import short_utils
from pathlib import Path

In [2]:
# globals
base_dir = short_utils.get_base_dir()
base_dir

PosixPath('~/lab/trans_stamp')

In [28]:
#print working dir
print(os.getcwd())

/home/shair/Desktop/STAMP_2023/jesse/trans_stamp_curr/scgpt/jesse_utils


In [3]:
def preview_file(bulk_genexp, num_rows = 3):

    #print first few elements of index, first column, colnames, and first 4x4
    print(f"first {num_rows}x{num_rows}: \n", bulk_genexp.iloc[0:num_rows, 0:num_rows])
    #print first 4 elements of index
    print("start of idx: \n", bulk_genexp.index[:(num_rows+1)])
    #print start of colnames
    print("start of colnames: \n", bulk_genexp.columns[:(num_rows+1)])

In [4]:
def get_obs(bulk_genexp,meta_cols):
    '''
    create the obs df containng the meta data about the samples
    :param bulk_genexp: the genexp data as a df
    :param meta_cols: a list of colnames with sample metadata
    :return: a df of the meta data
    '''
    obs_df = bulk_genexp[meta_cols]
    return obs_df

In [5]:
def add_cell_type(metadata_df,tcga_cancer):
    '''

    :param metadata: a df of sample meta data
    :param tcga_cancer: the cancer type of the bulk genexp data
    :return: add a 'cell_type" col with the cancer type
    '''
    metadata_df["cell_type"] = tcga_cancer
    return metadata_df

In [6]:
def add_metadata(file_path, metadata_df):
    '''

    :param file_path: path to table with additional metadata
    :param metadata_df: the metadata df
    :return: metadata_df with additional metadata
    '''
    pass

In [8]:
def get_var(genexp,gene_cols,hgnc_col):
    '''
    create the var df containing data about the variables, usually genes
    :param genexp: a df of genexp data
    :param gene_cols: a list of colnames with gene names
    :param hgnc_col: the colname with the hgnc gene names
    :return: a df of genes with col 'gene_hgnc'
    '''
    #subset the df to only the gene cols
    var_df = genexp[gene_cols]
    #rename hgnc_col gene_hgnc
    var_df = var_df.rename(columns={hgnc_col: "gene_hgnc"})
    #reorder cols and make hgnc the first col
    
    # Reorder columns to make 'gene_hgnc' the first column
    reordered_cols = ['gene_hgnc'] + [col for col in gene_cols if col != hgnc_col]
    var_df = var_df[reordered_cols]

    return var_df

# create an anndata object: .X is the counts, .var "gene_hgnc" is the genes, .obs "sample_id" is the sample_id and .obs "oncosig_label_ERBB2" is the oncosig_labels
adata = anndata.AnnData(X=counts, var=pd.DataFrame(index=genes, data=genes, columns=["gene_hgnc"]), obs=obs_df)


In [9]:
def create_anndata(counts_df, var_df,obs_df):
    '''
    :param counts_df: a df of counts with no names
    :param var_df: a df of genes with col 'gene_hgnc' as index
    :param obs_df: a df of sample meta data
    :return: an anndata object
    '''
    return anndata.AnnData(X=counts_df, var=var_df, obs=obs_df)


In [35]:
# def bulk_2_h5ad(bulk_genexp,tcga_cancer ='brca',metadata_cols = [],samples_metadata_path = '',\
#         genes_meta_path = '', save_name = ''):
#     #create the obs df: sample_id and oncosig_label_ERBB2 where the smaple id's are the idx
#     obs_df = get_sample_meta(bulk_genexp,metadata_cols)
#     #add a "cell_type" column to the obs_df with the value "brca"
#     obs_df = add_cell_type(obs_df,"brca")
#     #create the var and X df: gene_hgnc and the counts
#     #take subset, excluding the meta_data_cols
#     #drop meta_data_cols
#     bulk_genexp.drop(metadata_cols, axis=1, inplace=True)
# 
#     #save colnames to genes list
#     genes_var = get_genes(bulk_genexp)
#     #SAVE genexp counts to a counts df
#     counts = bulk_genexp.values
#     adata = create_anndata(counts, genes_var, obs_df)
# 
#     #save the anndata object
#     # raise err if no save name provided
#     if not save_name:
#         raise ValueError("no save name provided")
#     adata.write(base_dir + '/data/' +  save_name)



! run the script:
read the data, identify metadate cols, then create the anndata object

In [10]:
#set path to file
data_path = Path(base_dir / 'training_data/cell_lines/cell_line_states/hypoxia_acidosis_PMID_19057672/E-GEOD-9649-A-AFFY-44-normalized-expressions.tsv')
data_path

PosixPath('~/lab/trans_stamp/training_data/cell_lines/cell_line_states/hypoxia_acidosis_PMID_19057672/E-GEOD-9649-A-AFFY-44-normalized-expressions.tsv')

In [37]:
#read file: tab sepereated
bulk_genexp = pd.read_csv(data_path, sep='\t')

In [38]:
preview_file(bulk_genexp, 5)
#pritn colnames
print(bulk_genexp.columns)

first 5x5: 
    Unnamed: 0  TSPAN6 (7105)  TNMD (64102)  DPM1 (8813)  SCYL3 (57147)
0  ACH-001113       4.331992      0.000000     7.364660       2.792855
1  ACH-001289       4.567424      0.584963     7.106641       2.543496
2  ACH-001339       3.150560      0.000000     7.379118       2.333424
3  ACH-001538       5.085340      0.000000     7.154211       2.545968
4  ACH-000242       6.729417      0.000000     6.537917       2.456806
start of idx: 
 RangeIndex(start=0, stop=6, step=1)
start of colnames: 
 Index(['Unnamed: 0', 'TSPAN6 (7105)', 'TNMD (64102)', 'DPM1 (8813)',
       'SCYL3 (57147)', 'C1orf112 (55732)'],
      dtype='object')
Index(['Unnamed: 0', 'TSPAN6 (7105)', 'TNMD (64102)', 'DPM1 (8813)',
       'SCYL3 (57147)', 'C1orf112 (55732)', 'FGR (2268)', 'CFH (3075)',
       'FUCA2 (2519)', 'GCLC (2729)',
       ...
       'H3C2 (8358)', 'H3C3 (8352)', 'AC098582.1 (8916)',
       'DUS4L-BCAP29 (115253422)', 'C8orf44-SGK3 (100533105)',
       'ELOA3B (728929)', 'NPBWR1 (2831)'

In [39]:
#drop any chosen cols
bulk_genexp = bulk_genexp.drop(['DesignElementAccession'], axis=1)

In [40]:
#make necessary adjustments to the data
meta_data_cols = ['Gene ID', 'Gene Name']
save_name = "hypoxia_acidosis_PMID_19057672.h5ad"

In [41]:
#var_df 
var_df = get_var(bulk_genexp, ['Gene ID', 'Gene Name'], 'Gene Name')

In [42]:
#manually create obs_df
sample_ID = list(bulk_genexp.columns[2:])
#set cell type to be the string between first and last white space of each element in sample_ID:
#example: 'HMEC Lactic Acidosis plus hypoxia 2', 'HMEC Ctrl 1',

# Extracting cell type from each sample ID
cell_type = [' '.join(sample.split()[1:-1]) if len(sample.split()) > 2 else '' for sample in sample_ID]
#replcae Control with Ctrl
cell_type = [cell.replace('Control', 'Ctrl') for cell in cell_type]
obs_dic = {"sample_ID": sample_ID, "cell_type": cell_type}
obs_df = pd.DataFrame(obs_dic)


SyntaxError: invalid syntax (2913958980.py, line 1)

In [ ]:
#create counts df by removing the var cols and no colnames
counts_df = bulk_genexp.drop(['Gene ID', 'Gene Name'], axis=1).values.T

In [ ]:
#create anndata object
adata = create_anndata(counts_df, var_df, obs_df)

In [ ]:
#save the anndata object
save_path  = Path('/home/jesse/lab/trans_stamp/scgpt/data/trans_stamp_data/cell_lines/cell_line_states/hypoxia_acidosis_PMID_19057672/',  save_name)
save_path

In [ ]:
#write adata
adata.write(save_path)

! create meta data df

In [ ]:
'''
def bulk_2_h5ad(bulk_genexp,tcga_cancer ='brca',metadata_cols = [],samples_metadata_path = '',\
        genes_meta_path = '', save_name = ''):
'''
args = { 'bulk_genexp' : bulk_genexp,
        'tcga_cancer' : 'brca',
        'metadata_cols' : meta_data_cols,
        'samples_metadata_path' : '',
        'genes_meta_path' : '',
        'save_name' : save_name

}
bulk_2_h5ad(**args)

! create var and X df